In [ ]:
# ! conda remove torchvision pytorch torchaudio -y

In [ ]:
# ! conda install pytorch=2.0.1 torchvision=0.15.2 torchaudio pytorch::torchvision -c pytorch -y

In [39]:
import torch, torchvision
from torch import package
torchvision.disable_beta_transforms_warning()
import sys
import time
import modellib

In [40]:
if "__torch_package__" in dir():
    #inside a torch package
    import_func = package.PackageImporter().import_module
else:
    #normal
    import importlib
    import_func = lambda m: importlib.reload(importlib.import_module(m))

#internal modules
MODULES = ['datasets', 'traininglib']
[datasets, traininglib] = [import_func(m) for m in MODULES]

def save(self, destination):
    if isinstance(destination, str):
        destination = time.strftime(destination)
        if not destination.endswith('.pt.zip'):
            destination += '.pt.zip'
    try:
        import torch.package.package_importer as imp
        #re-export
        importer = (imp, torch.package.sys_importer)
    except ImportError as e:
        #first export
        importer = (torch.package.sys_importer,)

    with package.PackageExporter(destination, debug = True) as exp:
        interns = ['modellib']+MODULES
        exp.intern(interns)
        exp.extern('**', exclude=['torchvision.**'])
        externs = ['torchvision.ops.**', 'torchvision.datasets.**', 'torchvision.io.**', 'torchvision.models.*']
        exp.intern('torchvision.**', exclude=externs)
        exp.extern(externs, exclude='torchvision.models.detection.**')
        exp.intern('torchvision.models.detection.**')
        # force inclusion of internal modules + re-save if importlib.reload'ed
        for inmod in interns:
            if inmod in sys.modules:
                exp.save_source_file(inmod, sys.modules[inmod].__file__, dependencies=True)
            else:
                exp.save_source_string(inmod, importer[0].get_source(inmod))
        
        exp.save_pickle('model', 'model.pkl', self)
        exp.save_text('model', 'class_list.txt', '\n'.join(self.class_list))
    return destination

In [41]:
basemodel_pt_zip = save(modellib.DuckDetector(['AMCO', 'GADW', 'GWTE', 'MALL', 'NOPI', 'NSHO', 'REDH', 'RNDU']), 
                        "basemodel.pt.zip")

### <center>Test that packaged model opens</center>

In [42]:
torch.package.PackageImporter(basemodel_pt_zip).load_pickle('model', 'model.pkl', map_location='cpu')

ValueError: An entry is already registered under the name 'fasterrcnn_resnet50_fpn'.